# BƯỚC 1

# CRAWL DATA LAZADA

In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

In [ ]:
import numpy as np
from selenium import webdriver
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd

# driver = webdriver.Chrome(ChromeDriverManager().install())
driver = webdriver.Chrome()
driver.get("https://www.lazada.vn/catalog/?q=Qu%E1%BA%A7n%20sort%20nam")
sleep(random.randint(5, 10))

In [62]:
# get title and link of each item -> click title lay "RfADt"
elems = driver.find_elements(By.CSS_SELECTOR, ".RfADt [href]")
title = [elem.text for elem in elems]
links = [elem.get_attribute('href') for elem in elems]

In [63]:
# get link price of each item
elems_price = driver.find_elements(By.CSS_SELECTOR, ".aBrP0")
price = [elem_price.text for elem_price in elems_price]

df1 = pd.DataFrame(list(zip(title, price, links)), columns = ['title', 'price', 'link_item'])
df1['index_'] = np.arange(1, len(df1) + 1)

In [64]:
# Có item có discount, có item không có discount
elems_discount = driver.find_elements(By.CSS_SELECTOR, ".WNoq3")
discount = [elem.text for elem in elems_discount]

# elems_discount = driver.find_element(By.CSS_SELECTOR, ".WNoq3 .1m41m")
# discount = [elem.text for elem in elems_discount]

elems_discountPercent = driver.find_elements(By.CSS_SELECTOR, "WNoq3 .IcOsH")
discountPercent = [elem.text for elem in elems_discountPercent]

discount_list, discount_idx, discount_percent_list = [], [], []
for i in range(1, len(title) + 1):
    try:
        discount_percent = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[4]/span[1]".format(i))
        # print(discount_percent.text)
        discount_percent_list.append(discount_percent.text)
        discount_idx.append(i)
    except NoSuchElementException:
        print("No Such Element Exception " + str(i))

df2 = pd.DataFrame(list(zip(discount_idx, discount_percent_list)), columns = ['discount_idx', 'discount_percent_list'])
df3 = df1.merge(df2, how='left', left_on='index_', right_on='discount_idx')

df2.head(5)


No Such Element Exception 14
No Such Element Exception 39


,discount_idx,discount_percent_list
0,1,63% Off
1,2,50% Off
2,3,41% Off
3,4,31% Off
4,5,46% Off


In [65]:
df3.head(5)

,title,price,link_item,index_,discount_idx,discount_percent_list
0,Quần short unisex nam nữ chất cotton cao cấp c...,48.000 ₫,https://www.lazada.vn/products/quan-short-unis...,1,1.0,63% Off
1,"Quần short nam, quần ngắn chất kaki [LOẠI ĐẸP]...",59.999 ₫,https://www.lazada.vn/products/quan-short-nam-...,2,2.0,50% Off
2,Quần short sọt đùi ngắn thể thao nam 2 túi kéo...,41.000 ₫,https://www.lazada.vn/products/quan-short-sot-...,3,3.0,41% Off
3,Quần short nam thể thao 2 túi kéo khoá vải thu...,41.000 ₫,https://www.lazada.vn/products/quan-short-nam-...,4,4.0,31% Off
4,Quần đùi nam thể thao tập gym mặc nhà nhiều si...,29.900 ₫,https://www.lazada.vn/products/quan-dui-nam-th...,5,5.0,46% Off


In [66]:
# Có item có review, có item không có review
countReviews_list, countReviews_idx = [], []
for i in range(1, len(title) + 1):
    try:
        countReviews = driver.find_element("xpath", "/html/body/div[3]/div/div[2]/div[1]/div/div[1]/div[2]/div[{}]/div/div/div[2]/div[5]/div".format(i))
        countReviews_list.append(countReviews.text)
        countReviews_idx.append(i)
    except NoSuchElementException:
        print("No Such Element Exception " + str(i))

df4 = pd.DataFrame(list(zip(countReviews_idx, countReviews_list)), columns = ['countReviews_idx', 'countReviews_list'])
# join 2 dataframe
df5 = df3.merge(df4, how='left', left_on='index_', right_on='countReviews_idx')

No Such Element Exception 7


In [67]:
df5.head(5)

,title,price,link_item,index_,discount_idx,discount_percent_list,countReviews_idx,countReviews_list
0,Quần short unisex nam nữ chất cotton cao cấp c...,48.000 ₫,https://www.lazada.vn/products/quan-short-unis...,1,1.0,63% Off,1.0,(409)
1,"Quần short nam, quần ngắn chất kaki [LOẠI ĐẸP]...",59.999 ₫,https://www.lazada.vn/products/quan-short-nam-...,2,2.0,50% Off,2.0,(19)
2,Quần short sọt đùi ngắn thể thao nam 2 túi kéo...,41.000 ₫,https://www.lazada.vn/products/quan-short-sot-...,3,3.0,41% Off,3.0,(6548)
3,Quần short nam thể thao 2 túi kéo khoá vải thu...,41.000 ₫,https://www.lazada.vn/products/quan-short-nam-...,4,4.0,31% Off,4.0,(5128)
4,Quần đùi nam thể thao tập gym mặc nhà nhiều si...,29.900 ₫,https://www.lazada.vn/products/quan-dui-nam-th...,5,5.0,46% Off,5.0,(495)


In [68]:
# get link price of each item
elems_location = driver.find_elements(By.CSS_SELECTOR, ".oa6ri ")
location = [elem_location.text for elem_location in elems_location]

df5['location'] = pd.DataFrame(list(zip(location)), columns = ['location'])

In [69]:
df5.head()

,title,price,link_item,index_,discount_idx,discount_percent_list,countReviews_idx,countReviews_list,location
0,Quần short unisex nam nữ chất cotton cao cấp c...,48.000 ₫,https://www.lazada.vn/products/quan-short-unis...,1,1.0,63% Off,1.0,(409),Hà Nội
1,"Quần short nam, quần ngắn chất kaki [LOẠI ĐẸP]...",59.999 ₫,https://www.lazada.vn/products/quan-short-nam-...,2,2.0,50% Off,2.0,(19),Hồ Chí Minh
2,Quần short sọt đùi ngắn thể thao nam 2 túi kéo...,41.000 ₫,https://www.lazada.vn/products/quan-short-sot-...,3,3.0,41% Off,3.0,(6548),Hồ Chí Minh
3,Quần short nam thể thao 2 túi kéo khoá vải thu...,41.000 ₫,https://www.lazada.vn/products/quan-short-nam-...,4,4.0,31% Off,4.0,(5128),Hồ Chí Minh
4,Quần đùi nam thể thao tập gym mặc nhà nhiều si...,29.900 ₫,https://www.lazada.vn/products/quan-dui-nam-th...,5,5.0,46% Off,5.0,(495),Hồ Chí Minh


### Lấy thông tin chi tiết của từng item

Link 0 - page 1

In [70]:
links[8]

'https://www.lazada.vn/products/sale-50-quan-short-quan-lung-quan-sot-kaki-nam-chat-vai-kaki-mem-co-gian-nhe-quan-dui-ngo-nam-chat-kaki-cotton-di-bien-di-choi-dao-pho-i2102817722.html'

Lấy cho 1 trang đầu tiên

In [71]:
# # elems_star = driver.find_elements(By.CSS_SELECTOR, ".container-star starCtn left")
# # count_star = [elem.text for elem in elems_star]

# elems_date = driver.find_elements(By.CSS_SELECTOR, ".mod-reviews .item .top")
# date_comment = [elem.text for elem in elems_date]
# print(date_comment)

# elems_name = driver.find_elements(By.CSS_SELECTOR, ".mod-reviews .item .middle")
# name_comment = [elem.text for elem in elems_name]
# print(name_comment)

# elems_content = driver.find_elements(By.CSS_SELECTOR, ".mod-reviews .item .item-content .content")
# content_comment = [elem.text for elem in elems_content]
# print(content_comment)

# # elems_skuInfo = driver.find_elements(By.CSS_SELECTOR, ".mod-reviews .item-content .skuInfor")
# # skuInfo_comment = [elem.text for elem in elems_skuInfo]
# # print(skuInfo_comment)

# elems_likeCount = driver.find_elements(By.CSS_SELECTOR, ".mod-reviews .item .item-content .bottom .left .left-content")
# likeCount = [elem.text for elem in elems_likeCount]
# print(likeCount)

# df6 = pd.DataFrame(list(zip(date_comment, name_comment, content_comment, likeCount)), \
#                     columns = ['date_comment', 'name_comment', 'content_comment', 'like_count'])

# df6.insert(0, "link_item", links[0])

### Lấy comment từng page

In [72]:
pages = 8

In [73]:
links[pages]

'https://www.lazada.vn/products/sale-50-quan-short-quan-lung-quan-sot-kaki-nam-chat-vai-kaki-mem-co-gian-nhe-quan-dui-ngo-nam-chat-kaki-cotton-di-bien-di-choi-dao-pho-i2102817722.html'

In [74]:
driver.get(links[2])
sleep(100)

In [75]:
name_comment, content_comment, skuInfo_comment, like_count, star_count = [], [], [], [], []

ĐỌC 500 PAGE LƯU LẠI 1 LẦN

In [ ]:
count = 1

# # Tất cả sao
# next_pagination_cmt = driver.find_element("xpath", "/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/span[2]")
# next_pagination_cmt.click()
# sleep(2)

# # 5 sao
# next_pagination_cmt = driver.find_element("xpath", "/html/body/div[8]/div/div/ul/li[2]")
# next_pagination_cmt.click()

while True:
    try:
        print("Crawl Page " + str(count))
        # elems_date = driver.find_elements(By.CSS_SELECTOR, ".mod-reviews .item .top")
        # date_comment = [elem.text for elem in elems_date]

        elems_name = driver.find_elements(By.CSS_SELECTOR, ".mod-reviews .item .middle")
        name_comment = [elem.text for elem in elems_name] + name_comment

        elems_content = driver.find_elements(By.CSS_SELECTOR, ".item-content .content")
        content_comment = [elem.text for elem in elems_content] + content_comment

        elems_skuInfo = driver.find_elements(By.CSS_SELECTOR, ".mod-reviews .item-content .skuInfor")
        skuInfo_comment = [elem.text for elem in elems_skuInfo] + skuInfo_comment

        elems_likeCount = driver.find_elements(By.CSS_SELECTOR, ".mod-reviews .item .item-content .bottom .left .left-content")
        like_count = [elem.text for elem in elems_likeCount] + like_count

        stars_count = driver.find_elements(By.CSS_SELECTOR, ".condition")
        star_count = [elem.text for elem in stars_count] + star_count

        try:
            next_pagination_cmt = driver.find_element("xpath", "/html/body/div[4]/div/div[10]/div[1]/div[2]/div/div/div/div[3]/div[2]/div/button[2]")
            next_pagination_cmt.click()
            print("Clicked on button next page")
        except ElementNotInteractableException:
            print("Have no next page")

        # try:
        #     close_btn = driver.find_element("xpath", "/html/body/div[8]/div[2]/div")
        #     close_btn.click()
        #     print("Clicked on button exit!")
        #     sleep(random.randint(10))
        # except ElementNotInteractableException:
        #     count += 1
        #     continue
        count += 1
        sleep(random.randint(3, 10))
    except ElementNotInteractableException:
        print("Element Not Interactable Exceptional")
        break

In [15]:


# 5 sao
next_pagination_cmt = driver.find_element("xpath", "/html/body/div[8]/div/div/ul/li[2]")
next_pagination_cmt.click()

# 4 sao
next_pagination_cmt = driver.find_element("xpath", "/html/body/div[8]/div/div/ul/li[3]")
next_pagination_cmt.click()

# 3 sao
next_pagination_cmt = driver.find_element("xpath", "/html/body/div[8]/div/div/ul/li[4]")
next_pagination_cmt.click()

# 2 sao
next_pagination_cmt = driver.find_element("xpath", "/html/body/div[8]/div/div/ul/li[5]")
next_pagination_cmt.click()

# 1 sao
next_pagination_cmt = driver.find_element("xpath", "/html/body/div[8]/div/div/ul/li[6]")
next_pagination_cmt.click()

In [77]:
df6 = pd.DataFrame(list(zip(name_comment, content_comment, like_count, star_count)), \
                    columns = ['name_comment', 'content_comment', 'like_count', 'star_count'])

df6.insert(0, "link_item", links[pages])

In [ ]:
# df6 = pd.DataFrame(list(zip(name_comment, content_comment, like_count)), \
#                     columns = ['name_comment', 'content_comment', 'like_count'])

# df6.insert(0, "link_item", links[0])

In [58]:
df5.to_excel("quan_short_nam.xlsx")

In [78]:
df6.to_excel("quanshort_9.xlsx")

In [24]:
len(content_comment)

944

In [29]:
df6.head(5)

,link_item,name_comment,content_comment,like_count
0,https://www.lazada.vn/products/dien-thoai-sams...,Thu T.Chứng nhận đã mua hàng,"Giao hàng nhanh, hãng chính hãng, săn sale giá...",0
1,https://www.lazada.vn/products/dien-thoai-sams...,kioChứng nhận đã mua hàng,Cảm ơn bạn đã đánh giá cho Xiaomi! Chúng tôi r...,0
2,https://www.lazada.vn/products/dien-thoai-sams...,dương Q.Chứng nhận đã mua hàng,mâmmamamsmmsmsmmsmsmsmsmsmskskskksksksksksksjs...,0
3,https://www.lazada.vn/products/dien-thoai-sams...,tran N.Chứng nhận đã mua hàng,Cảm ơn bạn đã đánh giá cho Xiaomi! Chúng tôi r...,0
4,https://www.lazada.vn/products/dien-thoai-sams...,tran N.Chứng nhận đã mua hàng,"điện thoại như mô tả, mắt camera lòi nhưng chụ...",0


In [ ]:
driver.close()

In [1]:
pip install shopee-crawler

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for shopee-crawler: filename=shopee_crawler-0.2.3-py3-none-any.whl size=8853 sha256=59087ed381e25f1ad7d2c52a35dbb63ee19a1ac785419b66ac5594b4d6a935fa
  Stored in directory: c:\users\vinh.nt2\appdata\local\pip\cache\wheels\fd\f2\5c\956dafcc35fc372a4540504263ee25c5bf5f188c97e455418c
Successfully built shopee-crawler
Note: you may need to restart the kernel to use updated packages.


In [2]:
from shopee_crawler import Crawler

In [ ]:
crawler = Crawler()
crawler.set_origin(origin="shopee.vn") # Input your root Shopee website of your country that you want to crawl




data = crawler.crawl_by_search(keyword='điện thoại')



print(data)

### Chuẩn hóa kiểu chữ về cùng 1 dạng

In [25]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import json

url = 'https://shopee.vn/%C3%81o-Thun-%C4%90%C3%ADnh-N%C6%A1-Form-%C3%94m-Body-%C3%81o-Ph%C3%B4ng-Baby-Tee-Ch%E1%BA%A5t-Co-D%C3%A3n-i.454500388.25466637713?sp_atk=1245020a-67a2-4cd9-95eb-15f018c64020&xptdk=1245020a-67a2-4cd9-95eb-15f018c64020'
headers = {
'Host': 'shopee.vn',
'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:71.0) Gecko/20100101 Firefox/71.0',
'Cookie':'SPC_IA=-1; SPC_EC=-; SPC_F=L07IMDECRHjifEKyg7XuNCJ00GNdJGTA; REC_T_ID=246cfcdc-18fa-11ea-b254-f8f21e2be0b8; SPC_T_ID="Fyr1skVDq7FDiJOuTYHBmMfMr2Cw1eZyPbYJhBYoRmf/gvfvkOf5zgjIVXLrYYlg32aSx1PfmhWq7QsQzwM86mdeXG8VU7ERK4N+gfPFd14="; SPC_U=-; SPC_T_IV="/oJN8EB7iQwg7+n5mXd6cw=="; _gcl_au=1.1.788704691.1575727322; _fbp=fb.1.1575727322914.443117835; _ga=GA1.2.1422761069.1575727324; __BWfp=c1575727332595xf5a099d8b; cto_lwid=7ea874b3-f31f-47d7-aef9-60eed0156d33; cto_bundle=0tgQ7V9rU3JlRTU4aWlTc09JNXRaN014Y3ZXa1BtVVcwT2RhOU1UZ0tweUFvWUo2WHRPQjd0JTJCM1duaG5iWXFFRWxpbHZkTFluWUZLSEFudTFreGJueFoxU0EyanhnMWN6ZEVIUVV6cFlhd050emhFMWQ4bmhVelZwVSUyRmwwQUp5c29lOEhPT2ZobE10S1dvT09HYWNhVXV1YWx5R3dSOGw0MHcwZWpiZ2pXU2VHSzdrJTNE; _med=refer; G_ENABLED_IDPS=google; fbm_382498665271383=base_domain=.shopee.tw; SPC_SI=jq6hwq6ju6hig9hfulumcagdqaiopatc; _gid=GA1.2.143857303.1577796150; csrftoken=3Pya3o5WYEvhLOj9FqCqbV3angfwBlko; AMP_TOKEN=%24NOT_FOUND; _dc_gtm_UA-61915057-6=1'
}

r = requests.get(url,headers=headers,allow_redirects=True)
print(r.status_code)
print(r.history)
print(r.url)

driver = webdriver.Chrome()
driver.get(url)

soup = BeautifulSoup(r.text, 'html.parser')
items = soup.find_all("div", class_="col-xs-2-4 shopee-search-item-result__item")
print(len(items))

200
[]
https://shopee.vn/%C3%81o-Thun-%C4%90%C3%ADnh-N%C6%A1-Form-%C3%94m-Body-%C3%81o-Ph%C3%B4ng-Baby-Tee-Ch%E1%BA%A5t-Co-D%C3%A3n-i.454500388.25466637713?sp_atk=1245020a-67a2-4cd9-95eb-15f018c64020&xptdk=1245020a-67a2-4cd9-95eb-15f018c64020
0


In [24]:
discount_percent_list = []
for i in range(1, 7):
    try:
        discount_percent = driver.find_element("xpath", "/html/body/div[1]/div/div[2]/div[1]/div/div/div/div[2]/div[3]/div/div[1]/div[2]/div/div/div[3]/div[1]/div[{}]/div/div[3]".format(i))
        print(discount_percent.text)
        discount_percent_list.append(discount_percent.text)
        discount_idx.append(i)
    except NoSuchElementException:
        print("No Such Element Exception " + str(i))

No Such Element Exception 1
No Such Element Exception 2
No Such Element Exception 3
No Such Element Exception 4
No Such Element Exception 5
No Such Element Exception 6


In [23]:
print(discount_percent_list)

[]


In [31]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('start-maximized')
options.add_argument('disable-infobars')
options.add_argument('--disable-extensions')
browserdriver = webdriver.Chrome()
browserdriver.get('https://shopee.com.my/search?keyword=h370m')
WebDriverWait(browserdriver, 20).until(EC.element_to_be_clickable((By.XPATH, "//div[@class='shopee-modal__container']//button[text()='English']"))).click()
print([my_element.get_attribute('innerHTML') for my_element in WebDriverWait(browserdriver, 10).until(EC.presence_of_all_elements_located((By.XPATH, '//*[@class="_1NoI8_ _2gr36I"]')))])
print("Program Ended")

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF62CE8EEA2+31554]
	(No symbol) [0x00007FF62CE07ED9]
	(No symbol) [0x00007FF62CCC872A]
	(No symbol) [0x00007FF62CD18434]
	(No symbol) [0x00007FF62CD1853C]
	(No symbol) [0x00007FF62CD5F6A7]
	(No symbol) [0x00007FF62CD3D06F]
	(No symbol) [0x00007FF62CD5C977]
	(No symbol) [0x00007FF62CD3CDD3]
	(No symbol) [0x00007FF62CD0A33B]
	(No symbol) [0x00007FF62CD0AED1]
	GetHandleVerifier [0x00007FF62D198B1D+3217341]
	GetHandleVerifier [0x00007FF62D1E5AE3+3532675]
	GetHandleVerifier [0x00007FF62D1DB0E0+3489152]
	GetHandleVerifier [0x00007FF62CF3E776+750614]
	(No symbol) [0x00007FF62CE1375F]
	(No symbol) [0x00007FF62CE0EB14]
	(No symbol) [0x00007FF62CE0ECA2]
	(No symbol) [0x00007FF62CDFE16F]
	BaseThreadInitThunk [0x00007FFA99487374+20]
	RtlUserThreadStart [0x00007FFA9B33CC91+33]


# CRAWL DATA SHOPEE

In [9]:
id_item = '19787998337'
id_shop = '703090265'
page_50 = '1'

In [10]:
link_crawl = 'https://shopee.vn/api/v2/item/get_ratings?itemid=' + id_item + '&shopid=' + 'id_shop' + '&limit=50&offset=' + page_50

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
from selenium import webdriver
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
import json
# URL của API hoặc trang web

comments = pd.Series()

for i in range(1, 502):

    # Gửi yêu cầu GET
    driver = webdriver.Chrome()
    driver.get("https://my.lazada.vn/pdp/review/getReviewList?itemId=982542751&pageSize=50&filter=0&sort=0&pageNo={}".format(i))
    sleep(random.randint(5, 10))

    elems_content = driver.find_element("xpath", "/html/body/pre")
    print(elems_content.text)

    file_path = 'data.json'

    with open(file_path, 'w') as file:
        file.write(elems_content.text)

    # Mở file và đọc dữ liệu vào dictionary
    with open(file_path, 'r') as file:
        data = json.load(file)

    for i in data['data']['ratings']:
        comments[len(comments)] = i

In [85]:
comments

0      Lần đầu mua dùng thử, áo đẹp, form xinh săn sa...
1      Chất liệu:cotton\nMàu sắc:xanh tim than\nĐúng ...
2      Chất liệu:cotton\nMàu sắc:trắng\nĐúng với mô t...
3      Chất liệu:tốt\nMàu sắc:đẹp\nĐúng với mô tả:đún...
4      Đúng với mô tả:giống với ảnh mẫu\nChất liệu:vả...
                             ...                        
545    okila nhé mọi người giao nhanh đóng gói cẩn th...
546    Đúng với mô tả:xinhh\nMàu sắc:Xanh navi\n\nÁo ...
547    Màu sắc:naau,xam\nChất liệu:cotton\nĐúng với m...
548    Đúng với mô tả:dung\nChất liệu:vải cũng đc\nMà...
549    Màu sắc:trawsng\nĐúng với mô tả:đings\nChất li...
Length: 550, dtype: object

In [ ]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import numpy as np
from selenium import webdriver
from time import sleep
import random
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.by import By
import pandas as pd
import json
# URL của API hoặc trang web

driver = webdriver.Chrome()
driver.get("https://my.lazada.vn/pdp/review/getReviewList?itemId=982542751&pageSize=50&filter=0&sort=0&pageNo=1")
sleep(random.randint(5, 10))

elems_content = driver.find_element("xpath", "/html/body/pre")
print(elems_content.text)

file_path = 'data.json'

with open(file_path, 'w') as file:
    file.write(elems_content.text)

# Mở file và đọc dữ liệu vào dictionary
with open(file_path, 'r') as file:
    data = json.load(file)

comments = pd.Series()
for i in data['model']['items']:
    comments[len(comments)] = i

In [14]:
for i in data['model']:
    print(i, data['model'][i])

items [{'reviewRateId': 296100678542751, 'reviewType': 'PRODUCT_REVIEW', 'tradeOrderId': None, 'boughtDate': '09 thg 9 2021', 'zonedBoughtDate': None, 'reviewStatus': 'approved', 'reviewTitle': None, 'reviewContent': 'Giao hơi lâu\nNhưng giao đúng và đủ sản phẩm\nMua lazada mall nên rất an tâm\nCần là mua 👍👍👍👍👍', 'reviewContentList': [{'attribute': '', 'content': 'Giao hơi lâu\nNhưng giao đúng và đủ sản phẩm\nMua lazada mall nên rất an tâm\nCần là mua 👍👍👍👍👍', 'selected': None}], 'reviewTime': '21 thg 9 2021', 'zonedReviewTime': None, 'imageQCStatus': 'approved', 'images': [{'title': None, 'url': 'https://sg-test-11.slatic.net/other/roc/c5e2e4253428ecc5f814b49a8ac2a083.jpg', 'sort': 1, 'uploadTime': None, 'legal': True, 'status': 'approved', 'checkedByPeople': None, 'width': 2731, 'height': 2048, 'size': 563874}], 'videoId': 30029790743, 'isVideoReady': True, 'videoCoverUrl': 'https://vn-live-02.slatic.net/p/97baf0919f9180344d443cdbad511a84.jpg', 'reviewRateThreadId': None, 'mediaList':

In [ ]:
comments = pd.Series()

In [22]:
for i in data['model']['items']:
    comments[len(comments)] = i['reviewContent']

In [23]:
comments

0     Giao hơi lâu\nNhưng giao đúng và đủ sản phẩm\n...
1     Hàng mall chính hãng, giao hàng nhanh.. mua sa...
2     gdhd djhd jdud jdjd jdusj nshs jdhd jdud ndhd ...
3     Thành phần an toàn và tự nhiên, Nâng cao nấu ă...
4     Đóng gói tiện lợi với nhiều gói, Nâng cao hươn...
5     Đóng gói tiện lợi với nhiều gói, Thành phần an...
6     Một sản phẩm không thể thiếu trong mỗi căn bếp...
7     Hương vị ngon và thơm lừng, Một sản phẩm không...
8     Thành phần an toàn và tự nhiên, Thêm sâu sắc v...
9     Thành phần an toàn và tự nhiên, Hương vị ngon ...
10    Nâng cao hương vị của các công thức món ăn yêu...
11    Thêm sâu sắc và phong phú cho món ăn của bạn, ...
12    Lý tưởng cho những người quan tâm đến sức khỏe...
13    Nâng cao hương vị của các công thức món ăn yêu...
14    Đóng gói tiện lợi với nhiều gói, Lý tưởng cho ...
15    Nâng cao kỹ năng nấu nướng của bạn lên một tầm...
16    Nâng cao hương vị của các công thức món ăn yêu...
17    Nâng cao hương vị của các công thức món ăn

In [24]:
comments.to_excel('C:\\Users\\vinh.nt2\\Downloads\\BOT_NGOT_HAT_TO.xlsx')